In [83]:
import pymongo
import math 
# import re
import pandas as pd
# import numpy as np
import pylab 
from matplotlib import pyplot as plt
# from scipy import stats 
from  qgrid import show_grid as show 
from datetime import datetime
from powerapi.formula import RAPLFormulaHWPCReportHandler, FormulaState
from powerapi.report import HWPCReport


# import seaborn as sns

class testCase(object): 
    """in general its just the name of the machine where we launched the test """
    def __init__ (self , testname,database='rapls2',serveraddr='172.16.45.8',serverport=27017): 
        self._client = pymongo.MongoClient(serveraddr, serverport)
        self._db=self._client[database]
        self._testname=testname 
        self._sensors=self._db['sensor'+self._testname]
        self._formula_id = (None,)
        self._state = FormulaState(None, None, None, self._formula_id)
        self._handler = RAPLFormulaHWPCReportHandler(None)
        
    def gettimetamps(self,containername): 
        containerdata=list(self._sensors.find({'target':containername},projection=['timestamp']))
        begintime= containerdata[0]['timestamp']
        endtime=containerdata[-1]['timestamp']
        return begintime , endtime 

    def _process_power(self,row,socket,event):
        hwpc_report = HWPCReport.deserialize(row)
        for i in self._handler._process_report(hwpc_report, self._state) : 
            if i.metadata['socket']== socket and i.metadata['event']==event : 
                return i.power
        return -1 
    
    def _get_headers(self,row):
        hwpc_report = HWPCReport.deserialize(row)
        x=self._handler._process_report(hwpc_report,self._state)
        return [(i.metadata['socket'],i.metadata['event']) for i in x ]
    
    
    def getpowers(self,containername): 
        #get the power consumption of the system  between begin and end 
        begin , end = self.gettimetamps(containername) 
        x=list(self._sensors.find({'target':'all','timestamp' :{'$gte':begin,'$lte':end}}))
        conso= pd.DataFrame(x)

        headers=self._get_headers(x[0])
        for i in headers: 
            socket,event=i 
            title="powers_{}_{}".format(event.split('_')[-1],socket)
            conso[title]=conso.T.apply(lambda row: self._process_power(row,socket,event))

        return conso.drop(["_id","groups","sensor","target"],axis=1)
        
    def getpowersFromInterval(self,begin,end): 
        #get the power consumption of the system  between begin and end 
#         begin , end = self.gettimetamps(containername) 
        x=list(self._sensors.find({'target':'all','timestamp' :{'$gte':begin,'$lte':end}}))
        conso= pd.DataFrame(x)

        headers=self.get_headers(x[0])
        for i in headers: 
            socket,event=i 
            title="powers_{}_{}".format(event.split('_')[-1],socket)
            conso[title]=conso.T.apply(lambda row: self.process_power(row,socket,event))

        return conso.drop(["_id","groups","sensor","target"],axis=1)
   
    def getenergy(self,containername):
        powers =self.getpowers(containername)
        powers=powers.loc[:,[ i for i in powers.columns if 'powers_' in i ]]
        return powers.sum()    
    
    def getenergyfromInterval(self,begin,end):
        powers =self.getpowersFromInterval(begin,end)
        powers=powers.loc[:,[ i for i in powers.columns if 'powers_' in i ]]
        return powers.sum()
   
    def getrecap(self): 
        meausres = self._db['recap'+self._testname].find(projection={'_id': False,'id':False,'begin':False,'begin':False,'beginwarmup':False,'beginexecution':False,'end':False})
        return pd.DataFrame(list(meausres))

# simple example

In [84]:
#getting an instance of our machine 
machine = testCase("tt2080319")


In [90]:
machine.getenergy('tommti_python3_intArithmetic_50_0')

powers_DRAM_0     328.559555
powers_PKG_0     1494.400330
powers_DRAM_1     317.453461
powers_PKG_1      861.354065
dtype: float64

In [85]:
#to print the powers of a docker container named tommti_python3_intArithmetic_50_0
machine.getpowers('tommti_python3_intArithmetic_50_0')



,timestamp,powers_DRAM_0,powers_PKG_0,powers_DRAM_1,powers_PKG_1
0,2019-03-08 13:37:17.345,6.583572,29.674011,6.349091,14.447571
1,2019-03-08 13:37:18.345,6.575226,29.658569,6.336929,14.400757
2,2019-03-08 13:37:19.346,6.599869,29.647522,6.309570,14.386414
3,2019-03-08 13:37:20.346,6.613754,30.260437,6.380035,20.441345
4,2019-03-08 13:37:21.347,6.583984,29.666931,6.390656,15.033997
5,2019-03-08 13:37:22.347,6.603287,29.661865,6.310287,14.375549
6,2019-03-08 13:37:23.348,6.544464,29.661865,6.326187,14.394409
7,2019-03-08 13:37:24.348,6.573334,30.269043,6.392105,15.607910
8,2019-03-08 13:37:25.348,6.583008,30.379700,6.363968,21.900574
9,2019-03-08 13:37:26.349,6.595474,29.917480,6.333908,15.667175


In [89]:
#to print the energy of a docker container named  tommti_python3_intArithmetic_50_0
machine.getrecap()


,execution time,execution_powers_DRAM_0,execution_powers_DRAM_1,execution_powers_PKG_0,execution_powers_PKG_1,name,warmup time,warmup_powers_DRAM_0,warmup_powers_DRAM_1,warmup_powers_PKG_0,warmup_powers_PKG_1
0,40.0,264.231064,254.006790,1187.937561,746.708069,tommti_stacklesspython2_intArithmetic_50_0,4.0,26.458267,25.344406,115.548035,61.509033
1,50.0,329.764465,317.595154,1489.485046,872.288208,tommti_stacklesspython3_intArithmetic_50_0,5.0,32.972275,31.931122,143.381348,97.747681
2,39.0,257.802139,248.164749,1164.713928,705.609375,tommti_intelpython2_intArithmetic_50_0,3.0,19.770111,19.127167,78.928711,72.754028
3,57.0,375.262833,363.444992,1715.382568,924.901489,tommti_intelpython3_intArithmetic_50_0,6.0,39.646210,38.214966,167.352661,100.976624
4,52.0,343.344147,330.103073,1558.326477,856.672302,tommti_activepython_intArithmetic_50_0,6.0,38.577759,37.434280,157.176575,133.129333
5,18.0,118.961838,114.226074,561.123718,326.516663,tommti_cython2_intArithmetic_50_0,2.0,13.259949,12.813889,50.130066,41.945557
6,26.0,171.756638,165.317612,807.184631,439.785217,tommti_cython3_intArithmetic_50_0,3.0,18.915207,18.197754,85.690308,50.854065
7,2.0,13.874344,12.825668,58.955933,44.555664,tommti_graalpython_intArithmetic_50_0,1.0,5.793304,5.575409,21.488831,16.175598
8,25.0,165.529572,159.431717,762.991699,501.076660,tommti_ipy_intArithmetic_50_0,33.0,221.696075,211.201736,1016.345947,759.815430
9,15.0,124.904221,95.264038,456.951294,223.267761,tommti_jython_intArithmetic_50_0,3.0,20.454651,19.044373,89.003479,43.699036


In [68]:
pd.DataFrame().sum()


Series([], dtype: float64)